In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob

import pandas as pd

In [3]:
big_data = pd.DataFrame()
files = glob.iglob('backtests/binance/' + '**/backtest_result.txt', recursive=True)
i = 0
for filename in files:
    #print(f'parsing : {filename}')
    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': 'C:\\AgodaGit\\passivbot\\' + filename}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())

        #d['adgt_%'] = d['adgl_%']+d['adgs_%']
        big_data = big_data.append(d, ignore_index=True)

big_data

C:\Users\skhan\AppData\Local\Temp/ipykernel_235276/1315120660.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_235276/1315120660.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_235276/1315120660.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_235276/1315120660.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_

,file,v,s,ebrl,long,short,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrs,tgl_%,tgs_%
0,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.909573,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.114573,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.838780,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.952475,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.242079,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
678,C:\AgodaGit\passivbot\backtests/binance\XTZUSD...,5.3,XTZUSDT,0.841806,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
679,C:\AgodaGit\passivbot\backtests/binance\ZECUSD...,5.4,ZECUSDT,0.966600,True,True,0.0600,0.0179,100.0,100.0,0.9792,12.53,3.573
680,C:\AgodaGit\passivbot\backtests/binance\ZECUSD...,5.4,ZECUSDT,0.966600,True,True,0.0600,0.0179,100.0,100.0,0.9792,12.53,3.573
681,C:\AgodaGit\passivbot\backtests/binance\ZECUSD...,5.5,ZECUSDT,0.966000,True,True,0.0599,0.0173,100.0,100.0,0.9792,12.52,3.435


Now filter the dataframe

In [6]:
allowed_symbols = ["ETHUSDT", "XRPUSDT", "1000SHIBUSDT", "DOGEUSDT", "ADAUSDT",
                   "BNBUSDT", "MATICUSDT", "DOTUSDT", "SANDUSDT", "FTMUSDT",
                   "LTCUSDT", "LINKUSDT", "MANAUSDT", "ETCUSDT", "EOSUSDT",
                   "ATOMUSDT", "FILUSDT", "ICPUSDT", "ALICEUSDT", "LRCUSDT",
                   "BCHUSDT", "SUSHIUSDT", "XTZUSDT", "THETAUSDT", "CRVUSDT",
                   "ALGOUSDT", "CHZUSDT", "TRXUSDT", "VETUSDT", "CHRUSDT",
                   "ONEUSDT", "OMGUSDT", "XLMUSDT", "RUNEUSDT", "ENJUSDT",
                   "CELRUSDT", "SXPUSDT", "1INCHUSDT", "SRMUSDT", "COTIUSDT",
                   "ZECUSDT", "QTUMUSDT", "GRTUSDT", "DENTUSDT", "SFPUSDT",
                   "STORJUSDT", "IOTAUSDT", "HOTUSDT", "NEOUSDT", "COMPUSDT",
                   "LINAUSDT", "KAVAUSDT", "BATUSDT", "WAVESUSDT", "IOSTUSDT",
                   "ALPHAUSDT", "RLCUSDT", "HBARUSDT", "DODOUSDT", "XMRUSDT",
                   "SNXUSDT", "ANKRUSDT", "DASHUSDT", "REEFUSDT", "ZENUSDT",
                   "CVCUSDT", "RENUSDT", "ICXUSDT", "RSRUSDT", "SKLUSDT",
                   "NKNUSDT", "BELUSDT", "ONTUSDT", "OGNUSDT", "BLZUSDT",
                   "MTLUSDT", "CTKUSDT", "RVNUSDT", "LITUSDT", "MKRUSDT",
                   "OCEANUSDT", "ZILUSDT", "UNFIUSDT", "BANDUSDT", "YFIIUSDT",
                   "TOMOUSDT", "TRBUSDT", "KNCUSDT", "STMXUSDT", "ZRXUSDT",
                   "SCUSDT", "AKROUSDT", "XEMUSDT", "FLMUSDT", "BALUSDT",
                   "BTSUSDT", "DGBUSDT", "DEFIUSDT"]

allowed_symbols = ["OMGUSDT", "COTIUSDT", "ALICEUSDT","IOSTUSDT","ETCUSDT","GRTUSDT"]
big_data['adgt_%'] = big_data['adgl_%']+big_data['adgs_%']
df = big_data[(big_data.s.isin(allowed_symbols))
              & (big_data['long'] == True)
              & (big_data['short'] == True)
              & (big_data['adgt_%'] > 0.1 )
              & (big_data['bnkrl_%'] > 95.0)
              & (big_data['bnkrs_%'] > 95.0)
              & (big_data['v'] == '5.5')

    # & (big_data['tg_%'] > 10)
    # & (big_data['ebr'] > .85)
              ]
#df['total_adg'] = big_data['adgl_%'] + big_data['adgs_%']
df

,file,v,s,ebrl,long,short,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrs,tgl_%,tgs_%,adgt_%
89,C:\AgodaGit\passivbot\backtests/binance\ALICEU...,5.5,ALICEUSDT,0.9258,True,True,0.0319,0.0888,100.0,100.0,0.9159,6.505,18.770,0.1207
90,C:\AgodaGit\passivbot\backtests/binance\ALICEU...,5.5,ALICEUSDT,0.9380,True,True,0.0684,0.0703,100.0,100.0,0.9391,14.350,14.460,0.1387
91,C:\AgodaGit\passivbot\backtests/binance\ALICEU...,5.5,ALICEUSDT,0.9258,True,True,0.0319,0.0888,100.0,100.0,0.9159,6.505,18.770,0.1207
92,C:\AgodaGit\passivbot\backtests/binance\ALICEU...,5.5,ALICEUSDT,0.9380,True,True,0.0684,0.0703,100.0,100.0,0.9391,14.350,14.460,0.1387
93,C:\AgodaGit\passivbot\backtests/binance\ALICEU...,5.5,ALICEUSDT,0.9258,True,True,0.0319,0.0888,100.0,100.0,0.9159,6.505,18.770,0.1207
95,C:\AgodaGit\passivbot\backtests/binance\ALICEU...,5.5,ALICEUSDT,0.8934,True,True,0.0421,0.0890,100.0,100.0,0.8925,9.357,17.840,0.1311
96,C:\AgodaGit\passivbot\backtests/binance\ALICEU...,5.5,ALICEUSDT,0.9015,True,True,0.0367,0.1220,100.0,100.0,0.8606,8.464,24.300,0.1587
98,C:\AgodaGit\passivbot\backtests/binance\ALICEU...,5.5,ALICEUSDT,0.9180,True,True,0.0632,0.1190,100.0,100.0,0.8450,19.660,23.930,0.1822
201,C:\AgodaGit\passivbot\backtests/binance\COTIUS...,5.5,COTIUSDT,0.9508,True,True,0.0985,0.0654,100.0,100.0,0.8319,21.940,10.940,0.1639
202,C:\AgodaGit\passivbot\backtests/binance\COTIUS...,5.5,COTIUSDT,0.9508,True,True,0.0985,0.0654,100.0,100.0,0.8319,21.940,10.940,0.1639


In [7]:
df.drop_duplicates(subset=['s', 'tgl_%','tgs_%','adgl_%','adgs_%','adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',
                   inplace=True)
dx = df[['file','s','adgt_%','adgl_%','adgs_%','bnkrl_%', 'bnkrs_%','ebrl', 'ebrs']]
dx

C:\Users\skhan\AppData\Local\Temp/ipykernel_235276/2043545472.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=['s', 'tgl_%','tgs_%','adgl_%','adgs_%','adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',


,file,s,adgt_%,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs
89,C:\AgodaGit\passivbot\backtests/binance\ALICEU...,ALICEUSDT,0.1207,0.0319,0.0888,100.0,100.0,0.9258,0.9159
90,C:\AgodaGit\passivbot\backtests/binance\ALICEU...,ALICEUSDT,0.1387,0.0684,0.0703,100.0,100.0,0.9380,0.9391
95,C:\AgodaGit\passivbot\backtests/binance\ALICEU...,ALICEUSDT,0.1311,0.0421,0.0890,100.0,100.0,0.8934,0.8925
96,C:\AgodaGit\passivbot\backtests/binance\ALICEU...,ALICEUSDT,0.1587,0.0367,0.1220,100.0,100.0,0.9015,0.8606
98,C:\AgodaGit\passivbot\backtests/binance\ALICEU...,ALICEUSDT,0.1822,0.0632,0.1190,100.0,100.0,0.9180,0.8450
201,C:\AgodaGit\passivbot\backtests/binance\COTIUS...,COTIUSDT,0.1639,0.0985,0.0654,100.0,100.0,0.9508,0.8319
313,C:\AgodaGit\passivbot\backtests/binance\ETCUSD...,ETCUSDT,0.1004,0.0331,0.0673,100.0,100.0,0.9159,0.9393
314,C:\AgodaGit\passivbot\backtests/binance\ETCUSD...,ETCUSDT,0.1178,0.0379,0.0799,100.0,100.0,0.9465,0.9380
368,C:\AgodaGit\passivbot\backtests/binance\GRTUSD...,GRTUSDT,0.1158,0.0696,0.0462,100.0,100.0,0.9691,0.9847
370,C:\AgodaGit\passivbot\backtests/binance\GRTUSD...,GRTUSDT,0.1623,0.0639,0.0984,100.0,100.0,0.9163,0.9189
